In [136]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [100]:
df = pd.read_csv("CSV_Data_For_3.csv")

In [101]:
df.head(10)

,UserID,Order,Time
0,200000000.0,"[0,1,0]","[0,0,1]"
1,200000000.0,"[0,1,0]","[0,0,1]"
2,200000000.0,"[1,0,0]","[0,1,0]"
3,200000000.0,"[1,0,0]","[0,0,1]"
4,200000000.0,"[1,0,0]","[0,0,1]"
5,200000000.0,"[1,0,0]","[1,0,0]"
6,200199695.0,"[1,0,0]","[0,1,0]"
7,200921635.0,"[0,1,0]","[0,1,0]"
8,203011533.0,"[1,0,0]","[0,0,1]"
9,203051353.0,"[0,1,0]","[0,0,1]"


In [36]:
df.head()

,UserID,Order,Time
0,200000000.0,"[0,1,0]","[0,0,1]"
1,200000000.0,"[0,1,0]","[0,0,1]"
2,200000000.0,"[1,0,0]","[0,1,0]"
3,200000000.0,"[1,0,0]","[0,0,1]"
4,200000000.0,"[1,0,0]","[0,0,1]"


In [102]:
numpy_data = df.to_numpy()

In [103]:
sample_data = numpy_data[0:10]

In [104]:
for val in sample_data:
    val[1] = np.array(eval(val[1]))
    val[2] = np.array(eval(val[2]))
    print(val[2])

[0 0 1]
[0 0 1]
[0 1 0]
[0 0 1]
[0 0 1]
[1 0 0]
[0 1 0]
[0 1 0]
[0 0 1]
[0 0 1]


In [105]:
 # order_food_dict = {uniq: [[order, time], [order, time],   [order, time], [order, time]],   uniq2: [[order, time], [order, time],   [order, time], [order, time]]}

order_food_dict = {}
for sample in sample_data:
    #print(f"Key - {sample[0]}     Value - {' '.join([a for a in sample[1].split()[0:-1]])} {sample[2]}")
    #print("\n")
    key = sample[0]
    values = sample[1:]
    # check if the key is in the dictionary
    if order_food_dict.get(key) is not None:
         #   - get the value for that key
        existing_values = order_food_dict.get(key)
        # append new value to existing values
        existing_values.append(values)
        order_food_dict[key] = existing_values
        pass
    else:
        # create new values array
        new_values = []
        new_values.append(values)
        order_food_dict[key] = new_values
    
   
    pass

In [182]:
WINDOW_SIZE = 1
ORDER_SIZE = 3


new_dummy_data = []

for (k, v) in order_food_dict.items():
    customer_orders = order_food_dict.get(k)
    dummy_data = np.asarray(customer_orders)
    for j in range(len(dummy_data)):
        curr = list(dummy_data[j])
        if j + 1 < len(dummy_data):
            curr.append(dummy_data[j+1][1])
            pass
        else:
            curr.append([0,0,0])

        new_dummy_data.append(curr)
    

In [52]:
# customer_orders = order_food_dict.get(200000000.0)

In [13]:
# customer_orders

In [183]:
# new_dummy_data
x = []
y = []

In [184]:
for i in range(0, len(new_dummy_data)):
    if i + ORDER_SIZE < len(new_dummy_data):
        x.append(new_dummy_data[i : i+ORDER_SIZE])
        y.append(new_dummy_data[i+ORDER_SIZE][0])
#         print(f"Inputs -> {} ::::::: Output -> {new_dummy_data[i+ORDER_SIZE][0]} \n\n")

In [185]:
x = np.array(x)
y = np.array(y)

In [231]:
model = keras.Sequential()
model.add(keras.layers.Dense(64, input_shape=(3,3,3)))
model.add(keras.layers.Reshape((3, 64 * 3)))
model.add(tf.keras.layers.LSTM(4, return_sequences=True))
model.add(tf.keras.layers.LSTM(4))
model.add(tf.keras.layers.Dense(3))
model.add(tf.keras.layers.Dropout(0.2))
model.add(keras.layers.Activation('softmax'))

In [232]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["acc"])

In [233]:
model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 3, 3, 64)          256       
_________________________________________________________________
reshape_7 (Reshape)          (None, 3, 192)            0         
_________________________________________________________________
lstm_41 (LSTM)               (None, 3, 4)              3152      
_________________________________________________________________
lstm_42 (LSTM)               (None, 4)                 144       
_________________________________________________________________
dense_25 (Dense)             (None, 3)                 15        
_________________________________________________________________
dropout (Dropout)            (None, 3)                 0         
_________________________________________________________________
activation_4 (Activation)    (None, 3)               

In [168]:
np.array(y).shape

(7, 3)

In [234]:
model.fit(x, y, epochs=10, batch_size=2, validation_split=0.2) 

Epoch 1/10
3/3 [==============================] - 3s 288ms/step - loss: 1.0541 - acc: 1.0000 - val_loss: 1.0945 - val_acc: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 20ms/step - loss: 1.0479 - acc: 1.0000 - val_loss: 1.0878 - val_acc: 0.5000
Epoch 3/10
3/3 [==============================] - 0s 17ms/step - loss: 1.0238 - acc: 0.8375 - val_loss: 1.0821 - val_acc: 0.5000
Epoch 4/10
3/3 [==============================] - 0s 19ms/step - loss: 0.9595 - acc: 0.7125 - val_loss: 1.0766 - val_acc: 0.5000
Epoch 5/10
3/3 [==============================] - 0s 20ms/step - loss: 0.9476 - acc: 0.7125 - val_loss: 1.0715 - val_acc: 0.5000
Epoch 6/10
3/3 [==============================] - 0s 15ms/step - loss: 0.9339 - acc: 0.9000 - val_loss: 1.0669 - val_acc: 0.5000
Epoch 7/10
3/3 [==============================] - 0s 20ms/step - loss: 0.8355 - acc: 0.9000 - val_loss: 1.0628 - val_acc: 0.5000
Epoch 8/10
3/3 [==============================] - 0s 22ms/step - loss: 0.7633 - acc: 0.8375 - va

In [235]:
model.save("model.h5")

In [ ]:
model2 = keras